In [ ]:
datatype = 'test'
low_base = 0
high_base = 1000

In [ ]:
!python -m spacy download en_core_web_sm
!pip install amrlib
!pip install Penman

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 72.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.2/354.2 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 kB 4.3 MB/s eta 0:00:00
  Installing build dependencies ... - \ | / - \ | done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... - done
  Created wheel for smatch: filename=smatch-1.0.4-py3-none-any.whl size=24055 sha256=b3e189bdbde9526479b4bc792a32323f1d7f025eca2530e0969dd6e31557b78d
  Stored in directory: /root/.cache/pip/wheels/53/56/c6/da1826db6293069ea23fbff8f0a027939ee8087b0061281355
Successfully built smatch


In [ ]:
path_to_AMR3 = '**your path to AMR3.parsing.pt' #'/kaggle/input/amrpickledmodel/AMR3.parsing.pt'
!rm -r spring

rm: cannot remove 'spring': No such file or directory


In [ ]:
!git clone https://github.com/SapienzaNLP/spring.git

Cloning into 'spring'...
remote: Enumerating objects: 129, done.
remote: Counting objects: 100% (68/68), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 129 (delta 45), reused 44 (delta 27), pack-reused 61
Receiving objects: 100% (129/129), 504.09 KiB | 13.26 MiB/s, done.
Resolving deltas: 100% (60/60), done.


In [ ]:
!cd spring && pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 674.8/674.8 kB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 78.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 49.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=345697258256a764d2527c9fddd4ca6aedbd921dd8cd5cc22f333380c6874e03
  Stored in directory: /root/.cache/pip/wheels/5b/e0/77/05245143a5b31f65af6a21f7afd3219e9fa4896f918af45677
Successfully built sacremoses
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.2
    Uninstalling tokenizers-0.13.2:
      Successfully uninstalled tokenizers-0.13.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.26.1
    Uninstalling transformers-4.26.1:
      Successfully uninstalled transf

In [ ]:
!cd spring && pip install -e .

Obtaining file:///kaggle/working/spring
  Preparing metadata (setup.py) ... - done
  Running setup.py develop for spring-amr


In [ ]:
from pathlib import Path

import penman
import torch
from tqdm import tqdm
from spring.spring_amr import ROOT
from spring.bin.predict_amrs_from_plaintext import read_file_in_batches
from spring.spring_amr.evaluation import predict_amrs, compute_smatch
from spring.spring_amr.penman import encode
from spring.spring_amr.utils import instantiate_loader, instantiate_model_and_tokenizer

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [ ]:
with open('temp.txt', 'w') as f:
  f.write("Hi my name is John Doe I work for the AI tehcnology company, FNRH.")

In [ ]:
!ls

__notebook__.ipynb  spring  temp.txt


In [ ]:
args = {'texts': ['temp.txt'],
        'device': 'cuda',
        'model':'facebook/bart-large',
        'penman_linearization': True,
        'use_pointer_tokens': True,
        'raw_graph': False,
        'checkpoint': path_to_AMR3,
        'return_all': False,
        'use_recategorization': False,
        'batch_size': 1,
        'beam_size': 5}
args['only_ok'] = True
args['restore_name_ops'] = False

In [ ]:
if True:
    device = torch.device(args['device'])
    model, tokenizer = instantiate_model_and_tokenizer(
        args['model'],
        dropout=0.,
        attention_dropout=0,
        penman_linearization=args['penman_linearization'],
        use_pointer_tokens=args['use_pointer_tokens'],
    )
    model.load_state_dict(torch.load(args['checkpoint'], map_location='cpu')['model'])
    model.to(device)
    model.eval()

Downloading:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

In [ ]:
from datasets import load_dataset
dataset = load_dataset("ecthr_cases")

Generating train split:   0%|          | 0/9000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Dataset ecthr_cases downloaded and prepared to /root/.cache/huggingface/datasets/ecthr_cases/alleged-violation-prediction/1.1.0/8922a012792758e64921d4a66d42adf759e42838aae54a6a8871607f6399aecf. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
subset = dataset[datatype]['facts'][low_base:high_base]

In [ ]:
len(subset)

100

In [ ]:
import pandas as pd

In [ ]:
if True:
    for case_id, case in enumerate(tqdm(subset, desc='Cases:')):
        parsed_facts = []
        with tqdm(total=len(case)) as bar:
            for idx, string in enumerate(case):
                batch = [(idx, string, len(string.split()))]
                if not batch:
                    parsed_facts.append("")
                    continue
                ids, sentences, _ = zip(*batch)
                if len(sentences[0])<2000:
                    x, _ = tokenizer.batch_encode_sentences(sentences, device=device)
                    with torch.no_grad():
                        model.amr_mode = True
                        out = model.generate(**x, max_length=512, decoder_start_token_id=0, num_beams=args['beam_size'])
                else:
                    sentences_ = [sentences[0][i:i+2000] for i in range(0, len(sentences[0]), 2000)]
                    x, _ = tokenizer.batch_encode_sentences(sentences_, device=device)
                    with torch.no_grad():
                        model.amr_mode = True
                        out = model.generate(**x, max_length=512, decoder_start_token_id=0, num_beams=args['beam_size'])
                    out = torch.concatenate([o for o in out], axis=0).unsqueeze(0)


                bgraphs = []
                for idx, sent, tokk in zip(ids, sentences, out):
                    graph, status, (lin, backr) = tokenizer.decode_amr(tokk.tolist(), restore_name_ops=args['restore_name_ops'])
                    if args['only_ok'] and ('OK' not in str(status)):
                        continue
                    graph.metadata['status'] = str(status)
                    graph.metadata['source'] = 'path'
                    graph.metadata['nsent'] = str(idx)
                    graph.metadata['snt'] = sent
                    bgraphs.append((idx, graph))
                output = ""
                for i, g in bgraphs:
                    out = encode(g)
                    output += out+"\n"
                try:
                    output = output[output.index('# ::snt'):]
                    output = output[output.index('\n'):]
                except:
                    pass
                parsed_facts.append(output)
                bar.update(len(sentences))
        case_df = pd.DataFrame({"parsed": parsed_facts})
        case_df.to_csv("Case_"+str(low_base+case_id)+".csv", index=False)

 19%|█▉        | 4/21 [00:25<01:52,  6.65s/it]Building failure:
['(', '<pointer:0>', 'multi-sentence', ':li', 11, ':snt1', '(', '<pointer:1>', 'contrast-01', ':ARG2', '(', '<pointer:2>', 'allege-01', ':ARG0', '(', '<pointer:3>', 'person', ':ARG0-of', '(', '<pointer:4>', 'apply-01', ':ord', '(', '<pointer:5>', 'ordinal-entity', ':value', 1, ')', ')', ')', ':ARG1', '(', '<pointer:6>', 'have-purpose-91', ':ARG1', '(', '<pointer:7>', 'leave-11', ':ARG1', '(', '<pointer:8>', 'country', ':wiki', '"Yemen"', ':name', '(', '<pointer:9>', 'name', ':op1', '"Yemen"', ')', ')', ')', ':ARG2', '(', '<pointer:10>', 'protect-01', ':ARG0', '<pointer:3>', ':ARG1', '(', '<pointer:11>', 'person', ':ARG0-of', '(', '<pointer:12>', 'have-rel-role-91', ':ARG1', '<pointer:3>', ':ARG2', '(', '<pointer:13>', 'daughter', ')', ')', ')', ')', ':mod', '(', '<pointer:14>', 'main', ')', ')', ')', ')', ':snt2', '(', '<pointer:15>', 'and', ':op1', '(', '<pointer:16>', 'force-01', ':ARG1', '(', '<pointer:17>', 'person', '

In [ ]:
!mkdir downloads

In [ ]:
import os
import shutil

for file in [f for f in os.listdir() if '.csv' in f]:
    shutil.move(file, "./downloads/"+file)

In [ ]:
!zip -r downloads.zip ./downloads

  adding: downloads/ (stored 0%)
  adding: downloads/Case_6903.csv (deflated 88%)
  adding: downloads/Case_6915.csv (deflated 83%)
  adding: downloads/Case_6985.csv (deflated 88%)
  adding: downloads/Case_6981.csv (deflated 88%)
  adding: downloads/Case_6951.csv (deflated 87%)
  adding: downloads/Case_6923.csv (deflated 87%)
  adding: downloads/Case_6907.csv (deflated 88%)
  adding: downloads/Case_6992.csv (deflated 88%)
  adding: downloads/Case_6938.csv (deflated 87%)
  adding: downloads/Case_6921.csv (deflated 92%)
  adding: downloads/Case_6906.csv (deflated 87%)
  adding: downloads/Case_6904.csv (deflated 88%)
  adding: downloads/Case_6965.csv (deflated 89%)
  adding: downloads/Case_6996.csv (deflated 89%)
  adding: downloads/Case_6969.csv (deflated 87%)
  adding: downloads/Case_6958.csv (deflated 88%)
  adding: downloads/Case_6978.csv (deflated 89%)
  adding: downloads/Case_6956.csv (deflated 89%)
  adding: downloads/Case_6937.csv (deflated 87%)
  adding: downloads/Case_6908.csv (d